# URL dataset (ISCX-URL2016)

## Description
The Web has long become a major platform for online criminal activities. URLs are used as the main vehicle in this domain. To counter this issues security community focused its efforts on developing techniques for mostly blacklisting of malicious URLs.

While successful in protecting users from known malicious domains, this approach only solves part of the problem. The new malicious URLs that sprang up all over the web in masses commonly get a head start in this race. Besides that, Alexa ranked, trusted websites may convey compromised fraudulent URLs called defacement URL.

We explore a lightweight approach to detection and categorization of the malicious URLs according to their attack type and show that lexical analysis is effective and efficient for proactive detection of these URLs. We also study the effect of the obfuscation techniques on malicious URLs to figure out the type of obfuscation technique targeted at specific type of malicious URL. We study mainly five different types of URLs:

Benign URLs: Over 35,300 benign URLs were collected from Alexa top websites. The domains have been passed through a Heritrix web crawler to extract the URLs. Around half a million unique URLs are crawled initially and then passed to remove duplicate and domain only URLs. Later the extracted URLs have been checked through Virustotal to filter the benign URLs.

Spam URLs: Around 12,000 spam URLs were collected from the publicly available WEBSPAM-UK2007 dataset.

Phishing URLs: Around 10,000 phishing URLs were taken from OpenPhish which is a repository of active phishing sites.

Malware URLs: More than 11,500 URLs related to malware websites were obtained from DNS-BH which is a project that maintain list of malware sites.

Defacement URLs: More than 45,450 URLs belong to Defacement URL category. They are Alexa ranked trusted websites hosting fraudulent or hidden URL that contains both malicious web pages.

Obfuscation is used as a common method for masking malicious URLs. An attacker intending to evade static analysis on lexical URL features use obfuscation techniques so that malicious URLs become statistically like the benign ones. The obfuscation techniques on URLs is analyzed for the intent of malicious activity in this research. We analyzed mainly Spam, Phishing and Malware URLs to see what kind of obfuscation techniques applied on the URLs.

The full research paper outlining the details of the dataset and its underlying principles:

    Mohammad Saiful Islam Mamun, Mohammad Ahmad Rathore, Arash Habibi Lashkari, Natalia Stakhanova and Ali A. Ghorbani, "Detecting Malicious URLs Using Lexical Analysis", Network and System Security, Springer International Publishing, pp. 467-482, 2016.
[Download this dataset](https://www.unb.ca/cic/datasets/url-2016.html)

## Imports

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn. model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline

## Funciones auxiliares

In [8]:
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

In [9]:
# Representacion grafica del limite de decisión
def plot_svm_decision_boundary(svm_clf, xmin, xmax):
    w = svm_clf.coef_[0]
    b = svm_clf.intercedpt_[0]
    
    # At the decision boundary, w0 * x0 + w1 * x1 + b = 0
    # => x1 = -w0/w1 * x0 - b/w1
    
    x0 = np.linespace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]
    
    margin = 1/w[1]
    gutter_up = decision_boundary + margin
    gutter_down = decision_boundary - margin
    
    svs = svm_clf.support_vectors_
    plt.scaler(svs[:,0], s=180, facecolors='#FFAAAA')
    plt.plot(x, decision_boundary, "k-", linewidth = 2)
    plt.plot(x0, gutter_up, "k--", linewidth = 2)
    plt.plot(x0, gutter_down, "k--", linewidth = 2)
    

## 1.- Lectura del DataSet

In [11]:
df = pd.read_csv("./datasets/datasets/FinalDataset/Phishing.csv")

## 2.- Visualización preliminar de la información 

In [12]:
df.head(10)

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,ldl_domain,ldl_path,ldl_filename,ldl_getArg,dld_url,dld_domain,dld_path,dld_filename,dld_getArg,urlLen,domainlength,pathLength,subDirLen,fileNameLen,this.fileExtLen,ArgLen,pathurlRatio,ArgUrlRatio,argDomanRatio,domainUrlRatio,pathDomainRatio,argPathRatio,executable,isPortEighty,NumberofDotsinURL,ISIpAddressInDomainName,CharacterContinuityRate,LongestVariableValue,URL_DigitCount,host_DigitCount,Directory_DigitCount,File_name_DigitCount,Extension_DigitCount,Query_DigitCount,URL_Letter_Count,host_letter_count,Directory_LetterCount,Filename_LetterCount,Extension_LetterCount,Query_LetterCount,LongestPathTokenLength,Domain_LongestWordLength,Path_LongestWordLength,sub-Directory_LongestWordLength,Arguments_LongestWordLength,URL_sensitiveWord,URLQueries_variable,spcharUrl,delimeter_Domain,delimeter_path,delimeter_Count,NumberRate_URL,NumberRate_Domain,NumberRate_DirectoryName,NumberRate_FileName,NumberRate_Extension,NumberRate_AfterPath,SymbolCount_URL,SymbolCount_Domain,SymbolCount_Directoryname,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,2,12,5.5,8,4.083334,2,15,7,0,0,0,0,0,0,0,0,0,0,80,12,61,61,2,2,2,0.762500,0.025000,0.166667,0.150000,5.083334,0.032787,0,-1,1,-1,0.750000,-1,10,0,6,2,2,-1,54,11,0,0,39,-1,48,8,8,8,-1,0,0,5,0,7,-1,0.125000,0.0,-1.000000,-1.000000,-1.000000,-1.000000,6,1,-1,-1,-1,-1,0.676804,0.860529,-1.000000,-1.000000,-1.00000,-1.000000,benign
1,0,3,12,5.0,10,3.583333,3,12,8,2,0,2,2,0,0,0,0,0,0,78,17,54,54,40,4,2,0.692308,0.025641,0.117647,0.217949,3.176471,0.037037,0,-1,3,-1,0.647059,-1,8,0,6,2,0,-1,54,15,0,31,4,-1,40,10,8,7,-1,0,0,4,0,8,-1,0.102564,0.0,0.666667,0.044444,0.000000,-1.000000,7,2,2,1,0,-1,0.715629,0.776796,0.693127,0.738315,1.00000,-1.000000,benign
2,2,2,11,4.0,5,4.750000,2,16,11,0,0,0,0,0,0,0,0,0,0,71,9,54,54,2,2,7,0.760563,0.098592,0.777778,0.126761,6.000000,0.129630,0,-1,1,-1,0.666667,2,0,0,0,0,0,0,57,8,19,19,6,2,22,5,7,7,2,0,1,6,0,3,1,0.000000,0.0,0.000000,0.000000,NaN,0.000000,8,1,4,2,0,1,0.677701,1.000000,0.677704,0.916667,0.00000,0.898227,benign
3,0,2,7,4.5,7,5.714286,2,15,10,0,0,0,0,0,0,0,0,0,0,64,10,47,47,2,2,2,0.734375,0.031250,0.200000,0.156250,4.700000,0.042553,0,-1,1,-1,0.800000,-1,0,0,0,0,0,-1,53,9,5,5,30,-1,33,7,11,7,-1,0,0,4,0,3,-1,0.000000,0.0,0.000000,0.000000,NaN,-1.000000,5,1,2,0,0,-1,0.696067,0.879588,0.818007,0.753585,0.00000,-1.000000,benign
4,19,2,10,6.0,9,2.250000,2,9,5,0,0,0,0,0,0,0,0,0,0,68,13,48,48,2,2,35,0.705882,0.514706,2.692308,0.191176,3.692308,0.729167,0,-1,2,-1,0.769231,13,6,0,0,0,6,6,48,12,3,3,26,12,39,9,7,7,13,0,2,3,0,4,3,0.088235,0.0,0.000000,0.139535,0.153846,0.171429,9,1,1,5,4,3,0.747202,0.833700,0.655459,0.829535,0.83615,0.823008,benign
5,0,2,10,5.5,9,4.100000,2,15,11,0,0,0,0,0,0,0,0,0,0,69,12,50,50,44,4,2,0.724638,0.028986,0.166667,0.173913,4.166666,0.040000,0,-1,2,-1,0.833333,-1,6,0,-1,6,0,-1,50,11,-1,31,4,-1,44,9,7,7,-1,0,0,2,0,8,-1,0.086957,0.0,-1.000000,-1.000000,-1.000000,-1.000000,4,1,-1,-1,-1,-1,0.732981,0.860529,-1.000000,-1.000000,-1.00000,-1.000000,benign
6,0,2,12,4.5,6,5.333334,2,24,9,0,0,0,0,0,0,0,0,0,0,93,10,76,76,2,2,2,0.817204,0.021505,0.200000,0.107527,7.600000,0.026316,0,-1,1,-1,0.700000,-1,5,0,0,5,0,-1,72,9,6,0,53,-1,61,6,10,10,-1,0,0,4,0,8,-1,0.053763,0.0,0.357143,0.000000,NaN,-1.000000,5,1,2,0,0,-1,0.692383,0.939794,0.910795,0.673973,0.00000,-1.000000,benign
7,0,2,11,3.5,4,3.909091,2,15,6,0,0,0,0,0,0,0,0,0,0,68,8,53,53,2,2,2,0.779412,0.029412,0.250000,0.117647,6.625000,0.037736,0,-1,1,-1,0.625000,-1,0,0,-1,0,0,-1,54,7,-1,6,37,-1,45,4,9,9,-1,0,0,3,0,8,-1,0.000000,0.0,0.000000,0.000000,NaN,-1.000000,4,1,1,0,0,-1,0.707365,0.916667,0.916667,0.690332,0.00000,-1.000000,benign
8,0,2,9,2.5,3,4.555555,2,6,3,0,0,0,0,0,0,0,0,0,0,62,6,49,49,17,4,2,0.790323,0.032258,0.33333

In [13]:
df.describe()

/home/rebe/Proyectos/Simulacion/.venv/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,ldl_domain,ldl_path,ldl_filename,ldl_getArg,dld_url,dld_domain,dld_path,dld_filename,dld_getArg,urlLen,domainlength,pathLength,subDirLen,fileNameLen,this.fileExtLen,ArgLen,pathurlRatio,ArgUrlRatio,argDomanRatio,domainUrlRatio,pathDomainRatio,argPathRatio,executable,isPortEighty,NumberofDotsinURL,ISIpAddressInDomainName,CharacterContinuityRate,LongestVariableValue,URL_DigitCount,host_DigitCount,Directory_DigitCount,File_name_DigitCount,Extension_DigitCount,Query_DigitCount,URL_Letter_Count,host_letter_count,Directory_LetterCount,Filename_LetterCount,Extension_LetterCount,Query_LetterCount,LongestPathTokenLength,Domain_LongestWordLength,Path_LongestWordLength,sub-Directory_LongestWordLength,Arguments_LongestWordLength,URL_sensitiveWord,URLQueries_variable,spcharUrl,delimeter_Domain,delimeter_path,delimeter_Count,NumberRate_URL,NumberRate_Domain,NumberRate_DirectoryName,NumberRate_FileName,NumberRate_Extension,NumberRate_AfterPath,SymbolCount_URL,SymbolCount_Domain,SymbolCount_Directoryname,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath
count,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15096.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,1.536700e+04,15367.000000,15367.000000,15367.000000,15367.0,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.00000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15358.000000,15358.000000,8012.000000,15364.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,13541.000000,15177.000000,15364.000000,15364.000000
mean,3.446021,2.543698,8.477061,5.851956,10.027461,5.289936,2.543698,12.659986,8.398516,1.910913,0.141537,1.766187,0.259387,0.431900,0.850264,0.017310,0.832954,0.088241,0.217154,76.035401,16.600377,52.386347,52.386347,8.734171,2.601093,9.073404,0.643155,0.096030,0.696129,0.249828,4.004405,inf,0.000130,-0.999870,2.362010,-1.0,0.668651,1.971302,7.780439,0.411857,1.919438,1.513503,3.064424,0.320102,55.455652,14.499967,12.719659,8.627774,13.282423,1.191775,30.496909,8.857617,8.166265,7.096245,0.00449,0.025314,0.271165,4.164639,0.144205,3.774257,-0.389211,0.086108,0.021351,0.001530,0.072590,-0.108242,-0.819047,7.185137,1.543698,2.120843,1.124618,0.500813,-0.158782,0.721684,0.854232,0.634859,0.682896,0.313617,-0.723793
std,14.151453,0.944938,4.660250,2.064581,5.281090,3.535097,0.944938,8.562206,6.329007,4.657731,0.866726,4.561915,1.299338,3.039508,2.764976,0.391684,2.725000,0.688612,1.905833,33.448952,10.049775,34.001888,34.001888,17.120279,0.850948,22.355439,0.177744,0.169782,1.692045,0.154135,3.051601,NaN,0.011408,0.011408,1.781014,0.0,0.154372,12.827758,12.438931,1.833786,6.527359,4.169946,9.179328,6.603034,23.256247,8.338207,17.451959,11.176820,18.846877,8.342889,26.952844,4.294602,3.716196,3.873846,3.63047,0.162379,0.847973,1.975139,0.548710,4.035327,1.913854,0.100478,0.071968,0.336427,0.403709,0.387463,0.432338,3.542791,0.944938,2.777307,2.570246,2.261013,2.535939,0.049246,0.072641,0.510992,0.502288,0.576910,0.649785
min,0.000000,2.000000,0.000000,1.500000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.000000,4.000000,0.000000,0.000000,1.000000,1

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15367 entries, 0 to 15366
Data columns (total 80 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Querylength                      15367 non-null  int64  
 1   domain_token_count               15367 non-null  int64  
 2   path_token_count                 15367 non-null  int64  
 3   avgdomaintokenlen                15367 non-null  float64
 4   longdomaintokenlen               15367 non-null  int64  
 5   avgpathtokenlen                  15096 non-null  float64
 6   tld                              15367 non-null  int64  
 7   charcompvowels                   15367 non-null  int64  
 8   charcompace                      15367 non-null  int64  
 9   ldl_url                          15367 non-null  int64  
 10  ldl_domain                       15367 non-null  int64  
 11  ldl_path                         15367 non-null  int64  
 12  ldl_filename      

In [15]:
df["URL_Type_obf_Type"].value_counts()

URL_Type_obf_Type
benign      7781
phishing    7586
Name: count, dtype: int64

In [16]:
# Comprobar la existencial de valores nulos
is_null = df.isna().any()
is_null[is_null]

avgpathtokenlen             True
NumberRate_DirectoryName    True
NumberRate_FileName         True
NumberRate_Extension        True
NumberRate_AfterPath        True
Entropy_DirectoryName       True
Entropy_Filename            True
Entropy_Extension           True
Entropy_Afterpath           True
dtype: bool

In [18]:
# Comprobar la existencia de valores infiitos
is_inf = df.isin([np.inf, -np.inf]).any()
is_inf[is_inf]

argPathRatio    True
dtype: bool

In [19]:
# Representación grafica de dos características
plt.figure(figsize = (12, 6))

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>